### Zigbang 원룸 매물 데이터 수집

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 0. 라이브러리 불러오기

In [12]:
import pandas as pd
import requests

#### 1. 동이름으로 위도 경도 구하기

In [13]:
addr = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

response = requests.get(url)
print(response)

response.text

<Response [200]>


'{"success":true,"code":"200","items":[{"id":4002,"type":"address","name":"망원동","hint":"","description":"서울특별시 마포구 망원동","lat":37.556785583496094,"lng":126.9013442993164,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"서울시","local2":"마포구","local3":"망원동","web_level":15,"web_lat":37.556785583496094,"web_lng":126.9013442993164,"app_level":15,"app_lat":37.556785583496094,"app_lng":126.9013442993164,"법정동코드":"1144012300"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}}],"next":null,"limit":0}'

In [14]:
# 동이 여러개일경우에, 여러개 뜬다.
# 일단 편의를 위해 첫번째 값을 가져온다고 해보자.

data = response.json()['items'][0]
data

{'id': 4002,
 'type': 'address',
 'name': '망원동',
 'hint': '',
 'description': '서울특별시 마포구 망원동',
 'lat': 37.556785583496094,
 'lng': 126.9013442993164,
 'zoom': 5,
 'polygon': [],
 '_score': None,
 '_source': {'name_length': 3,
  'local1': '서울시',
  'local2': '마포구',
  'local3': '망원동',
  'web_level': 15,
  'web_lat': 37.556785583496094,
  'web_lng': 126.9013442993164,
  'app_level': 15,
  'app_lat': 37.556785583496094,
  'app_lng': 126.9013442993164,
  '법정동코드': '1144012300'},
 'zoom_level': {'google': 15, 'daum': 4},
 'zoom_level_v2': {'app': 5, 'web': 4}}

In [15]:
lat, lng = data['lat'], data['lng']
lat, lng 

(37.556785583496094, 126.9013442993164)

#### 2. 위도 경도로 geohash 알아내기
- geohash2 package 설치

In [16]:
!pip install geohash2

In [17]:
import geohash2

In [18]:
# presicion : 영역 크기 : 커질수록 영역이 작아진다.
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

#### 3. geohash로 매물 아이디 가져오기

In [19]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

response = requests.get(url)
print(response)

response.text

<Response [200]>


'{"clusters":[],"items":[{"lat":37.52957896797038,"lng":126.89981539511652,"item_id":37623131},{"lat":37.52910255446707,"lng":126.90042739831257,"item_id":37685329},{"lat":37.529597911464876,"lng":126.90041315587592,"item_id":37699264},{"lat":37.5295886669088,"lng":126.89958018247103,"item_id":37828417},{"lat":37.52960743464446,"lng":126.89758317220698,"item_id":37595863},{"lat":37.529116962216044,"lng":126.8969522519338,"item_id":37812771},{"lat":37.529692492203175,"lng":126.90731180728964,"item_id":37594373},{"lat":37.52975469741576,"lng":126.8994820019441,"item_id":37779193},{"lat":37.529807146389054,"lng":126.89735191205774,"item_id":37681821},{"lat":37.52978662830859,"lng":126.89796335208214,"item_id":37789981},{"lat":37.52930348427087,"lng":126.89734045105382,"item_id":37812120},{"lat":37.52939719075808,"lng":126.90756551290187,"item_id":37591840},{"lat":37.5299016829542,"lng":126.90755322990103,"item_id":37647361},{"lat":37.529375000863716,"lng":126.90690372443841,"item_id":3777

In [20]:
items = response.json()['items']
ids = [item['item_id'] for item in items]
len(ids), ids[:5]

(582, [37623131, 37685329, 37699264, 37828417, 37595863])

#### 4. 매물 아이디로 매물 정보 가져오기

In [21]:
url = f'https://apis.zigbang.com/v2/items/list'
params = {
    'domain' : 'zigbang',
    'item_ids' : ids}
response = requests.post(url, params)
response

<Response [200]>

In [22]:
response.json()

{'items': [{'item_id': 37623131,
   'section_type': None,
   'images_thumbnail': 'https://ic.zigbang.com/ic/items/37623131/1.jpg',
   'sales_type': '월세',
   'sales_title': '월세',
   'deposit': 1000,
   'rent': 50,
   'size_m2': 19.83,
   '공급면적': {'m2': 19.83, 'p': '6'},
   '전용면적': {'m2': 19.83, 'p': '6'},
   '계약면적': None,
   'room_type_title': None,
   'floor': '2',
   'floor_string': '2',
   'building_floor': '9',
   'title': '당산동에서 이보다 좋은 매물 못찾습니다',
   'is_first_movein': None,
   'room_type': '02',
   'status': True,
   'tags': ['추천'],
   'service_type': '원룸',
   'random_location': {'lat': 37.529089114452574, 'lng': 126.89977306882251},
   'manage_cost': '7',
   'reg_date': '2023-08-11T12:23:15+09:00',
   'is_new': False,
   'addressOrigin': {'local1': '서울시',
    'local2': '영등포구',
    'local3': '당산동4가',
    'local4': '',
    'fullText': '서울시 영등포구 당산동4가',
    'localText': '영등포구 당산동4가'},
   'action': {'isRead': False,
    'readAt': None,
    'isInquired': False,
    'inquiredAt': None,


In [23]:
data = response.json()['items']
df = pd.DataFrame(data)
df.tail(2)

,item_id,section_type,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_new,addressOrigin,action,contract,address,is_zzim,address1,address2,address3,item_bm_type
580,37703323,None,https://ic.zigbang.com/ic/items/37703323/1.jpg,월세,월세,5000,160,63.97,"{'m2': 63.97, 'p': '19.4'}","{'m2': 42.52, 'p': '12.9'}",...,False,"{'local1': '서울시', 'local2': '마포구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG
581,37836529,None,https://ic.zigbang.com/ic/items/37836529/1.jpg,월세,월세,5000,160,61.09,"{'m2': 61.09, 'p': '18.5'}","{'m2': 40.61, 'p': '12.3'}",...,True,"{'local1': '서울특별시', 'local2': '마포구', 'local3':...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG


In [24]:
# 데이터 프레임 볼 때 '...' 안나오게 하기
pd.options.display.max_columns= 40
pd.options.display.max_rows = 60

In [25]:
df.columns

Index(['item_id', 'section_type', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'status', 'tags', 'service_type',
       'random_location', 'manage_cost', 'reg_date', 'is_new', 'addressOrigin',
       'action', 'contract', 'address', 'is_zzim', 'address1', 'address2',
       'address3', 'item_bm_type'],
      dtype='object')

In [26]:
columns = ['item_id','sales_type', 'deposit', 'rent', 'size_m2', 
           'floor', 'floor_string','building_floor','title',
           'service_type','manage_cost','address1']

df = df[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,floor_string,building_floor,title,service_type,manage_cost,address1
580,37703323,월세,5000,160,63.97,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,빌라,8,서울시 마포구 중동
581,37836529,월세,5000,160,61.09,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,오피스텔,8,서울시 마포구 중동


In [27]:
# 망원동만 필터링하기
result = df[df['address1'].str.contains('망원동')].reset_index(drop=True)
result

,item_id,sales_type,deposit,rent,size_m2,floor,floor_string,building_floor,title,service_type,manage_cost,address1
0,37814767,월세,2000,45,23.14,1,1,3,3000.40 가능 . 풀옵션 . 큰 원룸 . 1.5층,원룸,5,서울시 마포구 망원동
1,37811154,월세,2000,45,23.14,1,1,3,리모델링원룸 빠른입주가능,원룸,5,서울시 마포구 망원동
2,37568534,전세,23000,0,44.29,2,2,2,가성비 따듯한 감성 리모델링 투룸,빌라,3,서울시 마포구 망원동
3,37626743,전세,28000,0,65.37,2,2,5,쓰리룸 빌라전세 HUG안심대출 주차가능,빌라,0,서울시 마포구 망원동
4,37814752,전세,27000,0,52.09,4,4,5,버팀목.보증보험가능. 테라스 있는 3룸이에요,빌라,3,서울시 마포구 망원동
5,37825096,전세,27000,0,52.89,4,4,5,망원역세권 테라스가 넓은 3룸 입니다.,빌라,3,서울시 마포구 망원동
6,37796306,전세,16000,0,41.21,2,2,3,가성비 주차 동물 버팀목 가능 투룸,빌라,1,서울시 마포구 망원동
7,37666663,월세,1000,75,23.14,2,2,4,망원동 핫플 리모델링 완료후 첫 입주,원룸,5,서울시 마포구 망원동
8,37709901,월세,1000,70,23.14,4,4,4,망원 신축 첫입주 풀옵션 리모델링 평수 채광 굳굳,원룸,5,서울시 마포구 망원동
9,37741462,전세,14000,0,26.45,2,2,4,잔여세대 1개ㅣ망원역5분ㅣ올리모델링ㅣ1.5룸,원룸,5,서울시 마포구 망원동


# 함수로 만들어서 활용해보기

In [28]:
import pandas as pd
import requests
import geohash2

In [34]:
def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    geohash = geohash2.encode(lat, lng, precision=5)
    
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    url = f'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain' : 'zigbang',
        'item_ids' : ids}
    response = requests.post(url, params)
    data = response.json()['items']
    df = pd.DataFrame(data)
    columns = ['item_id','sales_type', 'deposit', 'rent', 'size_m2', 
               'floor', 'floor_string','building_floor','title',
               'service_type','manage_cost','address1']

    df = df[columns]
    result = df[df['address1'].str.contains(addr)].reset_index(drop=True)
    return result

In [35]:
oneroom('망원동')

,item_id,sales_type,deposit,rent,size_m2,floor,floor_string,building_floor,title,service_type,manage_cost,address1
0,37814767,월세,2000,45,23.14,1,1,3,3000.40 가능 . 풀옵션 . 큰 원룸 . 1.5층,원룸,5,서울시 마포구 망원동
1,37811154,월세,2000,45,23.14,1,1,3,리모델링원룸 빠른입주가능,원룸,5,서울시 마포구 망원동
2,37568534,전세,23000,0,44.29,2,2,2,가성비 따듯한 감성 리모델링 투룸,빌라,3,서울시 마포구 망원동
3,37626743,전세,28000,0,65.37,2,2,5,쓰리룸 빌라전세 HUG안심대출 주차가능,빌라,0,서울시 마포구 망원동
4,37814752,전세,27000,0,52.09,4,4,5,버팀목.보증보험가능. 테라스 있는 3룸이에요,빌라,3,서울시 마포구 망원동
5,37825096,전세,27000,0,52.89,4,4,5,망원역세권 테라스가 넓은 3룸 입니다.,빌라,3,서울시 마포구 망원동
6,37796306,전세,16000,0,41.21,2,2,3,가성비 주차 동물 버팀목 가능 투룸,빌라,1,서울시 마포구 망원동
7,37666663,월세,1000,75,23.14,2,2,4,망원동 핫플 리모델링 완료후 첫 입주,원룸,5,서울시 마포구 망원동
8,37709901,월세,1000,70,23.14,4,4,4,망원 신축 첫입주 풀옵션 리모델링 평수 채광 굳굳,원룸,5,서울시 마포구 망원동
9,37741462,전세,14000,0,26.45,2,2,4,잔여세대 1개ㅣ망원역5분ㅣ올리모델링ㅣ1.5룸,원룸,5,서울시 마포구 망원동


In [31]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [ ]:
# 모듈 파일 만들기 : 변수, 함수, 클래스를 하나의 파일(.py)로 묶어서 코드 작성 실행 문법

In [38]:
%%writefile zigbang.py
import pandas as pd
import requests
import geohash2

def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    geohash = geohash2.encode(lat, lng, precision=5)
    
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    url = f'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain' : 'zigbang',
        'item_ids' : ids}
    response = requests.post(url, params)
    data = response.json()['items']
    df = pd.DataFrame(data)
    columns = ['item_id','sales_type', 'deposit', 'rent', 'size_m2', 
               'floor', 'floor_string','building_floor','title',
               'service_type','manage_cost','address1']

    df = df[columns]
    result = df[df['address1'].str.contains(addr)].reset_index(drop=True)
    return result

Writing zigbang.py


In [39]:
%ls zigbang.py

 C 드라이브의 볼륨: WINDOWS
 볼륨 일련 번호: 1CE2-C115

 C:\Users\user\Desktop\Kt_Study\Study\4.WebCrawling 디렉터리

2023-08-24  오후 05:02             1,200 zigbang.py
               1개 파일               1,200 바이트
               0개 디렉터리  171,557,662,720 바이트 남음
